# SPARQL Data Source Demo

Register the spark-fuse SPARQL data source, build the request payload, then read through
`spark.read.format` to execute a query against the Wikidata endpoint.

In [1]:
from spark_fuse.spark import create_session
from spark_fuse.io import (
    SPARQL_CONFIG_OPTION,
    SPARQL_DATA_SOURCE_NAME,
    build_sparql_config,
    register_sparql_data_source,
)

spark = create_session(app_name="sparql-demo")
register_sparql_data_source(spark)

:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/kevin/.ivy2.5.2/cache
The jars for the packages stored in: /Users/kevin/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b1fffad1-8b2f-4e09-9825-7d7325c78543;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 84ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-spark_2.13;4.0.0 from central in [default]
	io.delta#delta-storage;4.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.13.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules       

In [2]:
import json

sample_query = """
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>

SELECT ?pokemon ?pokemonLabel ?pokedexNumber WHERE {
  ?pokemon wdt:P31 wd:Q3966183 .
  ?pokemon wdt:P1685 ?pokedexNumber .
}
LIMIT 5
""".strip()

options = build_sparql_config(
    spark,
    "https://query.wikidata.org/sparql",
    source_config={
        "query": sample_query,
        "request_type": "POST",
        "headers": {"User-Agent": "spark-fuse-demo/1.0 (contact@example.com)"},
        "include_metadata": True,
    },
)

wikidata = (
    spark.read.format(SPARQL_DATA_SOURCE_NAME)
    .option(SPARQL_CONFIG_OPTION, json.dumps(options))
    .load()
)
wikidata.show(5, truncate=False)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py:161: DeprecationWarning: This process (pid=13553) is multi-threaded, use of fork() may lead to deadlocks in the child.


+-------+------------+-------------+
|pokemon|pokemonLabel|pokedexNumber|
+-------+------------+-------------+
+-------+------------+-------------+

